In [1]:
import math
from scripts_stock.cfg.out_file_name import OutFileName
from scripts_stock.cfg.set_dir import ProjectDir
from scripts_stock.cfg.stock_list import *
from scripts_stock.utils.analysis.stock_stat_index import df_to_stock_df, stock_kdj
import os
from scripts_stock.utils.common import CommonScript
from scripts_stock.utils.string_process import StringProcess
from scripts_stock.data_base.insert_into_db import insert_df_to_db,read_db_data
from scripts_stock.data_base.get_table_info import GetDataFromDB
from scripts_stock.data_base.db_py_sql import get_all_fuquan_one_stock
from scripts_stock.utils.logging_set import *


============ import stock list:  stock_list   ================


In [6]:
def get_stock_indicator(stock_df_in,if_print=False):
    stock_df = df_to_stock_df(stock_df_in[["open","close","high","low","stock_index","stock_date"]])
    stock_kdj_ind, _ = stock_kdj(stock_df)
    if if_print:
        print(stock_kdj_ind)
    #stock_kdj_ind["stock_index"] = stock_df_in["stock_index"].values[0]
    return stock_kdj_ind

In [89]:
def get_stock_indicator(target_db="test.db"):
    conn = CommonScript.connect_to_db(target_db)
    
    sql_query = f"SELECT * FROM t_stock_kdj_weekly_last60"
    df = pd.read_sql_query(sql_query, conn)
    conn.close()
    return df
df1 = get_stock_indicator()

In [110]:
df1

,date,open,close,high,low,stock_index,date_time,stock_date,macdh,cci,...,kdjk,kdjd,kdjj,boll_ub,boll_lb,macd,macds,wr_6,wr_10,update_time
0,2023-07-13,9.18,9.27,9.62,9.13,600011,2023-07-13,2023-07-13,-0.026,44.703,...,43.474,45.665,39.094,9.978,7.627,0.389,0.414,51.675,51.675,2024-09-30 03:02:08.708827
1,2023-07-20,9.22,8.87,9.25,8.86,600011,2023-07-20,2023-07-20,-0.051,-29.990,...,38.712,43.347,29.441,9.908,7.818,0.350,0.401,55.147,70.813,2024-09-30 03:02:08.708827
2,2023-07-27,8.87,8.31,9.10,8.20,600011,2023-07-27,2023-07-27,-0.104,-127.582,...,27.513,38.069,6.402,9.897,7.844,0.272,0.376,92.254,94.884,2024-09-30 03:02:08.708827
3,2023-08-03,8.25,8.82,8.88,8.21,600011,2023-08-03,2023-08-03,-0.102,-101.529,...,27.955,34.698,14.469,9.885,7.911,0.248,0.350,56.338,71.163,2024-09-30 03:02:08.708827
4,2023-08-10,8.80,8.64,9.15,8.60,600011,2023-08-10,2023-08-10,-0.111,-59.606,...,28.965,32.787,21.322,9.885,7.903,0.212,0.322,69.014,79.535,2024-09-30 03:02:08.708827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15055,2024-08-28,27.47,25.97,27.54,25.41,2460,2024-08-28,2024-08-28,0.070,-89.223,...,20.492,19.745,21.987,38.555,23.009,-3.485,-3.555,82.663,88.406,2024-09-30 03:02:08.708827
15056,2024-09-04,25.77,26.35,27.84,25.34,2460,2024-09-04,2024-09-04,0.128,-80.145,...,23.118,20.869,27.616,37.147,23.159,-3.396,-3.523,69.394,73.902,2024-09-30 03:02:08.708827
15057,2024-09-11,26.35,27.84,27.84,24.90,2460,2024-09-11,2024-09-11,0.284,-62.142,...,39.974,27.237,65.446,35.966,23.425,-3.168,-3.452,16.714,26.500,2024-09-30 03:02:08.708827
15058,2024-09-20,27.76,25.49,28.18,24.99,2460,2024-09-20,2024-09-20,0.249,-102.618,...,31.908,28.794,38.134,35.616,23.001,-3.141,-3.390,82.012,85.213,2024-09-30 03:02:08.708827


In [93]:
df_90 = df1.groupby('stock_index')[['kdjj','kdjd','kdjk','macdh','macd','macds']].quantile(0.9)

df1.groupby('stock_index')[['kdjj','kdjd','kdjk','macdh','macd','macds']].quantile(0.1)

,kdjj,kdjd,kdjk,macdh,macd,macds
stock_index,,,,,,
1,-0.1281,8.0202,6.4947,-0.0924,-0.6283,-0.5754
2,-0.4351,15.1753,12.2297,-0.1395,-1.0929,-1.0373
63,-5.5336,13.9620,11.6309,-1.1668,-2.3944,-2.1097
69,-4.6391,9.0805,7.2017,-0.0593,-0.4053,-0.3881
100,9.0236,19.4753,19.0296,-0.0723,-0.1147,-0.0341
...,...,...,...,...,...,...
603899,-8.4794,12.9024,10.0526,-0.6032,-2.5725,-2.2793
603986,-7.6724,18.5931,10.5711,-2.1717,-7.8174,-7.4002
603993,-0.1877,19.2845,15.5174,-0.2101,-0.1289,-0.0973


In [109]:
df_90 = df1.groupby('stock_index')[['kdjj','kdjd','kdjk','macdh','macd','macds']].quantile(0.9)
df_90.columns = [x+"_90" for x in df_90.columns]

df_10 = df1.groupby('stock_index')[['kdjj','kdjd','kdjk','macdh','macd','macds']].quantile(0.1)
df_10.columns = [x+"_10" for x in df_10.columns]

df_20 = df1.groupby('stock_index')[['kdjj','kdjd','kdjk','macdh','macd','macds']].quantile(0.2)
df_20.columns = [x+"_20" for x in df_20.columns]
df_out = df_90.reset_index().merge(df_10.reset_index()).merge(df_20.reset_index())
df_out

,stock_index,kdjj_90,kdjd_90,kdjk_90,macdh_90,macd_90,macds_90,kdjj_10,kdjd_10,kdjk_10,macdh_10,macd_10,macds_10,kdjj_20,kdjd_20,kdjk_20,macdh_20,macd_20,macds_20
0,1,86.4015,66.0745,67.5320,0.1869,0.1680,0.1235,-0.1281,8.0202,6.4947,-0.0924,-0.6283,-0.5754,4.9996,11.2150,10.8072,-0.0626,-0.4868,-0.4730
1,2,68.6654,39.0210,46.6295,0.1047,-0.5356,-0.5896,-0.4351,15.1753,12.2297,-0.1395,-1.0929,-1.0373,5.5486,17.2582,14.5688,-0.1166,-1.0100,-1.0090
2,63,81.9245,63.1167,67.0842,0.5943,2.0435,2.8343,-5.5336,13.9620,11.6309,-1.1668,-2.3944,-2.1097,-2.2232,15.4992,14.0786,-0.9418,-2.0838,-1.6892
3,69,72.4523,39.3773,48.8504,0.0341,-0.1726,-0.1689,-4.6391,9.0805,7.2017,-0.0593,-0.4053,-0.3881,-0.4054,10.9824,9.3826,-0.0498,-0.3564,-0.3592
4,100,90.3800,72.5809,76.6370,0.0551,0.1595,0.1504,9.0236,19.4753,19.0296,-0.0723,-0.1147,-0.0341,14.7634,27.9714,24.0318,-0.0504,0.0124,0.0268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,603899,93.7602,64.3732,71.0900,0.5561,-0.6289,-0.7252,-8.4794,12.9024,10.0526,-0.6032,-2.5725,-2.2793,-5.5636,20.2384,13.6830,-0.5244,-2.4154,-2.1614
247,603986,92.4568,79.3441,81.3127,2.3153,0.4786,0.1086,-7.6724,18.5931,10.5711,-2.1717,-7.8174,-7.4002,-2.1870,26.0152,20.2586,-1.7242,-6.1722,-6.1096
248,603993,101.0145,83.7544,85.9197,0.2839,0.8202,0.7343,-0.1877,19.2845,15.5174,-0.2101,-0.1289,-0.0973,5.7794,26.8828,22.0032,-0.1036,-0.0334,-0.0042
249,605117,92.3729,71.3255,73.5482,3.3664,7.1900,5.4823,-3.4934,10.2159,8.7666,-2.5087,-14.6851,-13.9869,6.6038,15.2760,12.1160,-0.7020,-13.9080,-12.6544


In [87]:
start_from = df_raw.shape[0]%5

3